# Лекция 11: "солянка"
*ни один суп не пострадал
1. Сокеты 
2. Flask(web server)
3. Jinja(шаблоны)
4. Парсинг(BeautifulSoup/lxml)
5. Сериализация

## Сокеты 
Socket(гнездо)- механизм общения нескольких процессов, позволяет общаться как процессам, находящимся на одной машине, так и удалённым. Бывают двух типов: серверные и клиентские. 

In [ ]:
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [ ]:
server_address = ('localhost', 10000)
sock.bind(server_address)

In [ ]:
sock.listen(1)

In [ ]:
def echo(sock):
    # tells we are in server mode
    sock.listen(1)

    while True:
        # Wait for a connection
        connection, client_address = sock.accept()
        try:
            print('connection from', client_address)

            # Receive the data in small chunks and retransmit it
            while True:
                data = connection.recv(16)
                print('received "%s"' % data)
                if data:
                    print('sending data back to the client')
                    connection.sendall(data)
                else:
                   print('no more data from', client_address)
                   break

        finally:
            # Clean up the connection
            connection.close()

In [ ]:
# server example
import socket
import sys
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_address = ('localhost', 10000)
sock.bind(server_address)
echo(sock)

In [ ]:
#client example
import socket
import sys

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = ('localhost', 10000)
print('connecting to %s port %s' % server_address)
sock.connect(server_address)

try:
    
    # Send data
    message = 'This is the message.  It will be repeated.'
    print('sending "%s"' % message)
    sock.sendall(message.encode('utf-8'))

    # Look for the response
    amount_received = 0
    amount_expected = len(message)
    
    while amount_received < amount_expected:
        data = sock.recv(16)
        amount_received += len(data)
        print('received "%s"' % data)

finally:
    print('closing socket')
    sock.close()

# Web frameworks 
Коротко: штуки, которые упрощают написание сайтов.

In [ ]:
import json 

from flask import Flask
from flask import Response, request

app = Flask(__name__)

@app.route('/hello',methods=['GET'])
def hello(): 
    return json.dumps({'msg' : 'hello my friend'}), 200

app.run(host='localhost', port=5000, use_reloader=False, threaded=True)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2017 20:15:49] "GET /hello HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 20:15:54] "GET /hello HTTP/1.1" 200 -


Давайте посмотрим, как что-нибудь интересное поделать

In [ ]:

import json 

from flask import Flask
from flask import Response, request

app = Flask(__name__)

@app.route('/hello',methods=['GET'])
def hello(): 
    return json.dumps({'msg' : 'hello my friend'}), 200

@app.route('/privet/<name>',methods=['GET'])
def hello2(name): 
    return json.dumps({'msg' : 'hello my friend,{}'.format(name)}), 200

def parse_bool(val): 
    return val.lower() == 'true'

@app.route('/bb/masha/<liters>',methods=['GET'])
def bb(name,liters): 
        return 'love you!', 200

@app.route('/bb/<name>/<liters>',methods=['GET'])
def bb(name,liters): 
    should_answer = request.args.get('need_answer', 'false')
    should_answer = parse_bool(should_answer)
    if should_answer: 
        return 'bb my friend',200
    else: 
        return 'How impolite!', 404

app.run(host='localhost', port=5000, use_reloader=False, threaded=True)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Nov/2017 19:41:36] "GET /hello/masha HTTP/1.1" 404 -
127.0.0.1 - - [29/Nov/2017 19:41:47] "GET /privet/masha HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 19:41:57] "GET /privet/masha HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 19:42:11] "GET /privet/masha?need_answer=false HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 19:42:26] "GET /bb/masha?need_answer=false HTTP/1.1" 404 -
127.0.0.1 - - [29/Nov/2017 19:42:34] "GET /bb/masha?need_answer=true HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 19:49:28] "GET /bb/masha/petia?need_answer=false HTTP/1.1" 404 -
127.0.0.1 - - [29/Nov/2017 19:55:10] "GET /bb/masha?need_answer=true HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2017 19:55:22] "POST /bb/masha?need_answer=true HTTP/1.1" 405 -
127.0.0.1 - - [29/Nov/2017 19:56:03] "POST /bb/masha?need_answer=true HTTP/1.1" 405 -


альтернативы django/falcon

# Jinja
Это про более удобную генерацию всяких разных html.

In [1]:
from jinja2 import Template
t = Template("Hello {{ something }}!")
t.render(something="World")

'Hello World!'

In [2]:
t = Template("My favorite numbers: {% for n in range(1,10) %}<b>{{n}}</b> ""{% endfor %}")
t.render()


'My favorite numbers: <b>1</b> <b>2</b> <b>3</b> <b>4</b> <b>5</b> <b>6</b> <b>7</b> <b>8</b> <b>9</b> '

Bigger example

In [ ]:
from flask import Flask, render_template
app = Flask(__name__)


@app.route("/")
def template_test():
    return render_template('template.html', my_string="Wheeeee!", my_list=[0,1,2,3,4,5])


if __name__ == '__main__':
    app.run(debug=True)

```
<!DOCTYPE html>
<html>
  <head>
    <title>Flask Template Example</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="http://netdna.bootstrapcdn.com/bootstrap/3.0.0/css/bootstrap.min.css" rel="stylesheet" media="screen">
    <style type="text/css">
      .container {
        max-width: 500px;
        padding-top: 100px;
      }
    </style>
  </head>
  <body>
    <div class="container">
      <p>My string: {{my_string}}</p>
      <p>Value from the list: {{my_list[3]}}</p>
      <p>Loop through the list:</p>
      <ul>
        {% for n in my_list %}
        <li>{{n}}</li>
        {% endfor %}
      </ul>
    </div>
    <script src="http://code.jquery.com/jquery-1.10.2.min.js"></script>
    <script src="http://netdna.bootstrapcdn.com/bootstrap/3.0.0/js/bootstrap.min.js"></script>
  </body>
</html>
```

```
{% include header.html%}
    <div class="container">
      <p>My string: {{my_string}}</p>
      <p>Value from the list: {{my_list[3]}}</p>
      <p>Loop through the list:</p>
      <ul>
        {% for n in my_list %}
        <li>{{n}}</li>
        {% endfor %}
      </ul>
    </div>
    <script src="http://code.jquery.com/jquery-1.10.2.min.js"></script>
    <script src="http://netdna.bootstrapcdn.com/bootstrap/3.0.0/js/bootstrap.min.js"></script>
  </body>
</html>
```

# Requests

In [4]:
import requests
r = requests.get('http://vk.com')

In [5]:
r.status_code

200

# Парсинг
2 альтернативы Beautifulsoup vs lxml 
Beautifulsoup - pure python - ставиться легко, но очень медленный. 
lxml - быстрый, но имеет биндинги к плюсам. 

In [2]:
import requests
from bs4 import BeautifulSoup
data = requests.get('http://bash.im').text

In [10]:
soup = BeautifulSoup(data, 'html.parser')
res = soup.findAll('div', attrs={'class': 'quote'})
for quote in res: 
    print(quote.text)



+ 589 –
[:||||:] 
2017-11-29 13:11 #447949 
xxx: Посмотрел в очередной раз "Хищника" и вдруг подумал:Для чего существу, которое видит только тепло, трофеи комнатной температуры?yyy:Может, он их подогревает перед тем, как насладиться просмотром своей коллекции?Что мы вообще знаем о причудах коллекционеров?



+ 1088 –
[:||||:] 
2017-11-29 12:46 #447948 
ххх: В метро передо мной какому-то студенту вместо одного жетона автомат высыпал весь запас, килограмма три. «Джекпот!» — радостным криком на всю станцию прокомментировали его товарищи. Что характерно, пошли сдавать обратно в кассу.yyy: По привычке пошли менять фишки на деньги.



+ 487 –
[:||||:] 
2017-11-29 12:12 #447947 
xxx: до 30 лет практически не употреблял,- а потом как-то незаметно втянулся.Бывало-работал на работе, и через каждый час (кафе напротив) бегал туда: 100 грамм и маленьким пивом запивал.В итоге за "трудовой день" -около литра водки и 3 литров пива.yyy: да тебе мЯдаль вручать надо за доблестное доползание с работы до

In [9]:
soup = BeautifulSoup(data, 'html.parser')
res = soup.findAll('div', attrs={'class': 'quote'})
for quote in res: 
    print(quote.find('div',attrs={'class':'text'}).text)
    break

xxx: Посмотрел в очередной раз "Хищника" и вдруг подумал:Для чего существу, которое видит только тепло, трофеи комнатной температуры?yyy:Может, он их подогревает перед тем, как насладиться просмотром своей коллекции?Что мы вообще знаем о причудах коллекционеров?


In [4]:
# libxml-dev
from lxml import etree
from io import StringIO, BytesIO
parser = etree.HTMLParser()
doc = etree.parse(StringIO(data),parser)
article_list = doc.findall("//div[@class='quote']")
#for elt in doc.iter('div'):
for ell in article_list:
    content = ell.find("div[@class='text']")
    print(etree.tostring(content))
    break

b'<div class="text">xxx: <br/>&#1055;&#1086;&#1089;&#1084;&#1086;&#1090;&#1088;&#1077;&#1083; &#1074; &#1086;&#1095;&#1077;&#1088;&#1077;&#1076;&#1085;&#1086;&#1081; &#1088;&#1072;&#1079; "&#1061;&#1080;&#1097;&#1085;&#1080;&#1082;&#1072;" &#1080; &#1074;&#1076;&#1088;&#1091;&#1075; &#1087;&#1086;&#1076;&#1091;&#1084;&#1072;&#1083;:<br/>&#1044;&#1083;&#1103; &#1095;&#1077;&#1075;&#1086; &#1089;&#1091;&#1097;&#1077;&#1089;&#1090;&#1074;&#1091;, &#1082;&#1086;&#1090;&#1086;&#1088;&#1086;&#1077; &#1074;&#1080;&#1076;&#1080;&#1090; &#1090;&#1086;&#1083;&#1100;&#1082;&#1086; &#1090;&#1077;&#1087;&#1083;&#1086;, &#1090;&#1088;&#1086;&#1092;&#1077;&#1080; &#1082;&#1086;&#1084;&#1085;&#1072;&#1090;&#1085;&#1086;&#1081; &#1090;&#1077;&#1084;&#1087;&#1077;&#1088;&#1072;&#1090;&#1091;&#1088;&#1099;?<br/><br/>yyy:<br/>&#1052;&#1086;&#1078;&#1077;&#1090;, &#1086;&#1085; &#1080;&#1093; &#1087;&#1086;&#1076;&#1086;&#1075;&#1088;&#1077;&#1074;&#1072;&#1077;&#1090; &#1087;&#1077;&#1088;&#1077;&#1076; &

# Сериализация
Обычно сериализуют в человеко читабельный текст(json), однако есть сериализация в бинарный формат(pickle). 
Pickle используется при общении через Queue. У него есть некотороу число ограничений:не сериализует лямбды, для десериализации обьект должены быть "importable" от корня. Для этого есть ещё dill. 

Это "рабоче-крестьянские" сериализации, есть ещё protobuf(можете самостоятельно почитать, что это такое)


In [3]:
import json 
d = {1:2}
serialized = json.dumps(d)
print(serialized)
d2 = json.loads(serialized)
print(d2)

{"1": 2}
{'1': 2}


In [ ]:
class A(object): 
    def __init__(self,val): 
        self.val = val

In [8]:
import pickle
#cPickle

In [10]:
a = A(42)

In [11]:
pickle.dumps(a)

b'\x80\x03c__main__\nA\nq\x00)\x81q\x01}q\x02X\x03\x00\x00\x00valq\x03K*sb.'

In [12]:
pickled = pickle.dumps(a)

In [13]:
a2 =pickle.loads(pickled)

In [14]:
a2.val

42

In [15]:
a2.val+=1

In [16]:
a.val

42

In [17]:
pickle.dumps(lambda x: x+a.val)

PicklingError: Can't pickle <function <lambda> at 0x7f65d0807d08>: attribute lookup <lambda> on __main__ failed